In [2]:
# imports for libraries and local modules
import main
import numpy as np
import json
import networkx as nx
import itertools
import re
import matplotlib
import csv
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# The big question
## Does the CS department belong more to the college of science or the college of engineering?

The way we will do this is by running multiple community detection algorithms and then comparing the comunnti that have The CS department science departments and engineering departments. Going off of multiple scores and found communities that we can find which college the CS department 

### community detection algorithms to run

In [5]:
Gf2022, Gs2023, G_reg = main.create_graphs()
Gf2022.size()
# nx.draw(Gf2022)
l = list(Gf2022.nodes())
for i in l:
    if i.startswith('s'): Gf2022.remove_node(i)
l = list(Gf2022.nodes())
for i in l:
    if i.startswith('g'): Gf2022.remove_node(i)
l = list(Gf2022.nodes())
# for i in l:
#     if i.startswith('d'): Gf2022.remove_node(i)
# for i in nx.adjacency_matrix(G_reg):
#     print(i)
# for i in nx.connected_components(Gf2022):
#     print(i)


### removing isolated nodes

In [6]:
# remove isolated nodes
l = [i for i in list(Gf2022.nodes()) if len(list(Gf2022.neighbors(i))) == 0]
print(len(list(Gf2022.nodes())))
for i in l:
    Gf2022.remove_node(i)
print(len(list(Gf2022.nodes())))

7680
4859


In [145]:
########### useful lists
dlist = [i for i in list(Gf2022.nodes()) if i.startswith('d')]

### testing to see if colleges will end up in their own comunities or others
(takes too long 40 min+)

In [ ]:
# gn_com = nx.community.girvan_newman(Gf2022)
# coms = []
# for com in itertools.islice(gn_com,1):
#     coms.append(com)
# len(coms)

In [146]:
sim_rank = nx.simrank_similarity(Gf2022, tolerance=.05)

clustering functions based on node similarity

In [168]:
def merge_2_in_list(lst, i:set, j:set):
    lst.remove(i)
    lst.remove(j)
    comb = set.union(i, j)
    lst.append(comb)
    return lst

def setsim(sim_l, x, y):
    most = 0
    for i in x:
        for j in y:
            if sim_l[i][j] > most:
                most = sim_l[i][j]
    return most

def merge_best(slist, sim_l):
    most = 0 
    best = ({},{})
    for x in slist:
        for y in slist:
            if not x == y:
                if setsim(sim_l, x, y) > most:
                    best = (x,y)
                    most = setsim(sim_l, x, y)
    slist = merge_2_in_list(slist, best[0], best[1])
    return slist

In [169]:
list(sim_rank.keys())[500]

dsets = [{i} for i in dlist]

while len(dsets) > 35:
    dsets = merge_best(dsets, sim_rank)
    # for s in dsets:
    #     print(s)
    # print()
    for s in dsets:
        if 'dComputer Science' in s:
            print(s)
for i in dlist:
    print(i, sim_rank['dComputer Science'][i])


{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dComputer Science'}
{'dMathematics and Statistics', 'dComputer Science', 'dCollege Graduate Degree Programs', 'dBiology', 'dWatershed Sciences', 'dElectrical and Computer Engineering', 'dWildland Resources', 'dGeosciences', 'dEnvironment and Society', 'dPhysics', 'dMechanical and Aerospace Engineering', 'dChemistry and Biochemistry'}
{'dMathematics and Statistics', 'dComputer Science', 'dCollege Graduate Degree Programs', 'dBiology', 'dWatershed Sciences', 'dElectrical and Computer Engineering', 'dWildland Resources', 'dGeosciences', 'dEnvironment and Society', 'dBiological Engineering', 'dPhysics', 'dMechanical and Aerospace Engineering', 'dChemistry and Biochemistry'}
dNone 0.0
d

Computer science first unions with
{'dMathematics and Statistics', 'dComputer Science', 'dCollege Graduate Degree Programs', 'dBiology', 'dWatershed Sciences', 'dElectrical and Computer Engineering', 'dWildland Resources', 'dGeosciences', 'dEnvironment and Society', 'dPhysics', 'dMechanical and Aerospace Engineering', 'dChemistry and Biochemistry'}
departments CS is most similar to.

Mathematics and Statistics 0.01756198138117592
Electrical and Computer Engineering 0.013817100358897143
Physics 0.012728859470477173
Mechanical and Aerospace Engineering 0.00894455550982796
Geosciences 0.008916532749064133
Plants, Soils and Climate 0.007701980279592437
Chemistry and Biochemistry 0.0075383358937803875
Biology 0.007196324834269024
Biological Engineering 0.006135921440022809
Sociology and Anthropology 0.005347494805756185

### Common Neighbor Centrality

In [210]:
cnc_sim = nx.common_neighbor_centrality(Gf2022, alpha=.5)

In [211]:
# for i in cnc_sim:
#     print(i)
cnc_lst = []
for i in cnc_sim:
    if i[0].startswith('d') and i[1].startswith('d'):
        cnc_lst.append(i)

#### functions to combine a list for tuples instead of dictionaries

In [212]:
def setsim2(sim_l, x, y):
    most = 0
    for i in x:
        for j in y:
            for k in sim_l:
                if k[0] == i and k[1] == j:    
                    if k[2] > most:
                        most = k[2]
    return most

def merge_best2(slist, sim_l):
    most = 0 
    best = ({},{})
    for x in slist:
        for y in slist:
            if not x == y:
                if setsim2(sim_l, x, y) > most:
                    best = (x,y)
                    most = setsim2(sim_l, x, y)
    slist = merge_2_in_list(slist, best[0], best[1])
    return slist

In [213]:
for i in cnc_lst:
    if i[0] == 'dComputer Science':
        print(i)

('dComputer Science', 'dMilitary Science', 404.9166666666667)
('dComputer Science', 'dJournalism and Communication', 607.375)
('dComputer Science', 'dBiology', 607.375)
('dComputer Science', 'dApplied Economics', 607.375)
('dComputer Science', 'dArt and Design', 607.375)
('dComputer Science', 'dSchool of Teacher Education and Leadership', 607.375)
('dComputer Science', 'dWorld Languages and Cultures', 607.375)
('dComputer Science', 'dElectrical and Computer Engineering', 607.375)
('dComputer Science', 'dEnvironment and Society', 607.375)
('dComputer Science', 'dEngineering Education', 0.0)
('dComputer Science', 'dManagement', 607.375)
('dComputer Science', 'dCommunicative Disorders and Deaf Education', 607.375)
('dComputer Science', 'dData Analytics and Information Systems', 607.375)
('dComputer Science', 'dNursing', 607.375)
('dComputer Science', 'dEnglish', 607.375)
('dComputer Science', 'dCollege Graduate Degree Programs', 404.9166666666667)
('dComputer Science', 'dChemistry and Bio

CS highest similarities according to common_neighbor_centrality
interestingly enough more than 70% of departments shared the highest similarity score in this centraility measure.

In [214]:
dsets2 = [{i} for i in dlist]

while len(dsets2) > 20:
    dsets2 = merge_best2(dsets2, cnc_lst)

for i in dsets2:
    print(i)
    

{'dNone'}
{'dEngineering Education'}
{'dAerospace Studies'}
{'dCenter for Anticipatory Intelligence (CAI)'}
{'dMathematics and Statistics', 'dHuman Development and Family Studies'}
{'dKinesiology and Health Science', 'dInstructional Technology and Learning Sciences'}
{'dNursing', 'dPsychology'}
{'dSpecial Education and Rehabilitation Counseling', 'dSchool of Teacher Education and Leadership'}
{'dData Analytics and Information Systems', 'dEconomics and Finance'}
{'dMarketing and Strategy', 'dManagement'}
{'dSchool of Accountancy', 'dArt and Design', 'dMusic'}
{'dWildland Resources', 'dEnvironment and Society'}
{'dCollege Graduate Degree Programs', 'dWatershed Sciences'}
{'dTheatre Arts', 'dApplied Economics', 'dAnimal, Dairy and Veterinary Sciences', 'dAviation Technology'}
{'dNutrition, Dietetics and Food Sciences', 'dApplied Sciences, Technology and Education', 'dMechanical and Aerospace Engineering', 'dLandscape Architecture and Environmental Planning'}
{'dTechnology, Design and Tech

A lot of these groupings do no make sense i believe that the degree centrality obscures how these nodes interact comparing common neighbors

In [231]:
clique_lst = []
genclique = nx.community.k_clique_communities(Gf2022, 3)
for i in genclique:
    if True in (t.startswith('d') for t in i):
        clique_lst.append(i)

for i in clique_lst:
    print(i)


[]


In [8]:
Gf2022, Gs2023, G_reg = main.create_graphs()
Gf2022.size()
# nx.draw(Gf2022)
l = list(Gf2022.nodes())
for i in l:
    if i.startswith('s'): Gf2022.remove_node(i)
l = [i for i in list(Gf2022.nodes()) if len(list(Gf2022.neighbors(i))) == 0]
print(len(list(Gf2022.nodes())))
for i in l:
    Gf2022.remove_node(i)
print(len(list(Gf2022.nodes())))

bruh = nx.panther_similarity(Gf2022, 'dComputer Science', 20)
print(bruh)

7689
4868
{'p33679': 0.00040114325828611544, 'p34264': 0.00036771465342893915, 'p33683': 0.00028414314128599846, 'p33684': 0.00028414314128599846, 'p33681': 0.0002674288388574103, 'p34334': 0.0002507145364288221, 'gCollege of Science': 0.000234000234000234, 'p33682': 0.000234000234000234, 'p34260': 0.00018385732671446958, 'cCS 6970': 0.00010028581457152886, 'cCS 3200': 8.357151214294072e-05, 'cCS 1030': 8.357151214294072e-05, 'cPSC 6150': 6.685720971435257e-05, 'cCS 5850': 6.685720971435257e-05, 'cSOC 6150': 6.685720971435257e-05, 'cCS 5250': 6.685720971435257e-05, 'cCS 7990': 6.685720971435257e-05, 'cCS 7950': 6.685720971435257e-05, 'cCS 6250': 6.685720971435257e-05}
